In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Đọc dữ liệu
df_r = pd.read_csv("GlobalTemperatures.csv", parse_dates=["dt"])
df = df_r[df_r["dt"] >= "1850-01-01"].copy(deep=False)

# Thêm cột năm (numeric) từ datetime
df["year"] = df["dt"].dt.year
df["month"] = df["dt"].dt.month
# Hoặc có thể dùng timestamp: df["timestamp"] = df["dt"].astype("int64") // 10**9
df.head()
# Chọn cột số
numeric_cols = df.select_dtypes(include=["float64", "int32"])

# Tính ma trận tương quan
corr_matrix = numeric_cols.corr()

# In ra ma trận
print("Ma trận tương quan giữa các thuộc tính (có year):")
print(corr_matrix)

# Vẽ heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(
    corr_matrix,
    annot=True,          # hiển thị số trên ô
    cmap="coolwarm",     # màu sắc xanh - đỏ
    fmt=".2f",           # format 2 chữ số thập phân
    cbar=True
)
plt.title("Heatmap - Ma trận tương quan ", fontsize=14)
plt.show()

# Xử lý dữ liệu ở phía bên dưới, bên trên là mô hình cũ

In [ ]:
import pandas as pd
df = pd.read_csv("GlobalTemperatures.csv", parse_dates=["dt"])
# df

### Thêm cột "year" và xóa những bản ghi trước năm 1850

In [ ]:
df["year"] = df["dt"].dt.year
df = df[df["year"] >=1850]
# df.head()

### Xóa cột "dt"

In [ ]:
df = df.drop(columns="dt")

# df.head()

### Nhóm các thuộc tính còn lại theo "year"

In [ ]:
group = df.groupby("year")[df.drop(columns="year").columns].mean()
# group

### Chuẩn hóa dữ liệu bằng sklearn

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

feature_cols = ['LandAverageTemperature', 'LandAverageTemperatureUncertainty',
       'LandMaxTemperature', 'LandMaxTemperatureUncertainty',
       'LandMinTemperature', 'LandMinTemperatureUncertainty',
       'LandAndOceanAverageTemperature',
       'LandAndOceanAverageTemperatureUncertainty']

X = group[feature_cols]

temperature_features = ['LandAverageTemperature', 'LandAverageTemperatureUncertainty',
       'LandMaxTemperature', 'LandMaxTemperatureUncertainty',
       'LandMinTemperature', 'LandMinTemperatureUncertainty',
       'LandAndOceanAverageTemperature',
       'LandAndOceanAverageTemperatureUncertainty']

# Tạo ColumnTransformer để xử lý từng phần
preprocessor = ColumnTransformer(transformers=[
    ('temp_scaler', StandardScaler(), temperature_features)
])

# X

### Biến đổi dữ liệu thành dataFrame có 6 thuộc tính: Nam_1 ... Nam_6
#### Mỗi cell là một vecto có 8 phần tử 
#### Nam_6 là biến mục tiêu

In [ ]:
import numpy as np
preProcess = {
    "Nam_1":[],
    "Nam_2" : [],
    "Nam_3":[],
    "Nam_4" : [],
    "Nam_5": [],
    "Nam_6" : []
}


# for Nam in preProcess.keys():
vectos =[]    
for index, row in group.iterrows():
    vecto = np.array([])
    for item in row:
        vecto = np.append(vecto, item)
    vectos.append(vecto)
    
i = 0         
for index in range(len(vectos)):
    for p in range(index, index+6):
        Nam = f"Nam_{i%6+1}"
        preProcess[Nam].append((list)(vectos[p]))
        i+=1
    if index == len(vectos)-7:
        break

# print(type(preProcess["Nam_1"][1]))
# print(preProcess["Nam_6"][0])


In [ ]:
new_df = pd.DataFrame(preProcess)
# new_df

### X là feature, Y là biến mục tiêu

In [ ]:
X = new_df[['Nam_1', 'Nam_2', 'Nam_3', 'Nam_4', 'Nam_5']]
y = new_df['Nam_6']

# X
# y

### Chia tập train, test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# X_train
# y_train

### Sử dụng học máy truyền thống
#### Đổi X,Y sang array numpy thì mới huấn luyện được
#### Vì mỗi cell ở X là 1 vecto nên khó huân luyện nên dùng sum(row,[]) để nối toàn bộ vecto trong 1 hàng của X từ đó ta có ma trận 2D X và Y

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Chuyển đổi X_train, X_test, y_train, y_test từ list sang numpy array cho từng sample
def convert_X(X_df: pd.DataFrame):
    # Mỗi cell là 1 list, ghép lại thành 2D array
    return np.array([np.array(sum(row, [])) for row in X_df.values])

def convert_y(y_series: pd.Series):
    return np.array([np.array(y) for y in y_series.values])

X_train_np = convert_X(X_train)
X_test_np = convert_X(X_test)
y_train_np = convert_y(y_train)
y_test_np = convert_y(y_test)

# Huấn luyện mô hình Linear Regression
lr = LinearRegression()
lr.fit(X_train_np, y_train_np)

# Dự đoán trên tập test
y_pred = lr.predict(X_test_np)

# Đánh giá mô hình (ví dụ: MSE trung bình trên từng chiều)
mse = mean_squared_error(y_test_np, y_pred)
r2 = r2_score(y_test_np, y_pred)

print("Mean Squared Error:", mse)
print("R2 Score:", r2)

#### Biểu đồ thể hiện đầu ra thực tế và đầu ra dự đoán, đường kẻ đỏ là đường thẳng gần giống với đầu ra thực tế vì có xu hướng tăng (có thể là đường cong nhưng để trực quan thì để thành đường thẳng tính min - max)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))
for i in range(8):  # vì có 8 output
    plt.subplot(2, 4, i+1)
    plt.scatter(y_test_np[:, i], y_pred[:, i], alpha=0.6)
    plt.plot([y_test_np[:, i].min(), y_test_np[:, i].max()],
             [y_test_np[:, i].min(), y_test_np[:, i].max()],
             'r--')
    plt.xlabel("Thực tế")
    plt.ylabel("Dự đoán")
    plt.title(f"Output {i+1}")
plt.tight_layout()
plt.show()

In [ ]:
X_train_np[0]

In [ ]:
y_train_np[0]

### Sử dụng học sâu để làm

In [ ]:
import shutil
import numpy as np
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, InputLayer

def r2_score(y_true, y_pred):
    ss_res = K.sum(K.square(y_true - y_pred))  # Residual sum of squares
    ss_tot = K.sum(K.square(y_true - K.mean(y_true)))  # Total sum of squares
    return 1 - ss_res / (ss_tot + K.epsilon())

# Chuyển đổi X, y thành numpy array 3D/2D
def convert_X_3d(X_df: pd.DataFrame):
    # Mỗi hàng là list 5 vector (mỗi vector 8 chiều)
    return np.array([np.stack(row) for row in X_df.values])

def convert_y_2d(y_series: pd.Series):
    return np.array([np.array(y) for y in y_series.values])

X_train_3d = convert_X_3d(X_train)
X_test_3d = convert_X_3d(X_test)
y_train_2d = convert_y_2d(y_train)
y_test_2d = convert_y_2d(y_test)

# Xây dựng mô hình
model = Sequential([
    InputLayer(shape=(5, 8)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(8)  # 8 output
])

model.compile(optimizer='adam', loss='mse', metrics=['mae', r2_score])
model.fit(X_train_3d, y_train_2d, epochs=100, batch_size=8, verbose=0)

y_pred_dl = model.predict(X_test_3d)
# Đánh giá
loss, mae, r2 = model.evaluate(X_test_3d, y_test_2d, verbose=0)
print("Test Loss:", loss)
print("Test MAE:", mae)
print("Test R²:", r2)
# print("X:", X_train_3d)

model.save("planetheat_model.keras")
shutil.copy("planetheat_model.keras", "../backend/model.keras")

y_pred_dl

In [ ]:
# X_test_3d[-1]
X_test[-5:]

#### Biểu đồ thể hiện đầu ra thực tế và đầu ra dự đoán, đường kẻ đỏ là đường thẳng gần giống với đầu ra thực tế vì có xu hướng tăng (có thể là đường cong nhưng để trực quan thì để thành đường thẳng tính min - max)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))
for i in range(8):  # vì có 8 output
    plt.subplot(2, 4, i+1)
    plt.scatter(y_test_2d[:, i], y_pred_dl[:, i], alpha=0.6)
    plt.plot([y_test_2d[:, i].min(), y_test_2d[:, i].max()],
             [y_test_2d[:, i].min(), y_test_2d[:, i].max()],
             'r--')
    plt.xlabel("Thực tế")
    plt.ylabel("Dự đoán")
    plt.title(f"Output {i+1}")
plt.tight_layout()
plt.show()